# 452 - Machine Learning - HW8 - Digit Classification using Tensorflow & Amazon Sagemaker

The task is to understand the concept of deep learning and how it can be easily deployed as a running model using Amazon Sagemaker. I have created a notebook instance on sagemaker and an S3 bucket for the data to be stored on the cloud. The rest of the code has been leveraged entirely from the Tensorflow sample notebook provided on Sagemaker console.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-up-the-environment" data-toc-modified-id="Set-up-the-environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set up the environment</a></span></li><li><span><a href="#Download-the-MNIST-dataset" data-toc-modified-id="Download-the-MNIST-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Download the MNIST dataset</a></span></li><li><span><a href="#Upload-the-data" data-toc-modified-id="Upload-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Upload the data</a></span></li><li><span><a href="#Custom-function-for-Tensorflow" data-toc-modified-id="Custom-function-for-Tensorflow-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Custom function for Tensorflow</a></span></li><li><span><a href="#Create-a-training-job-using-the-sagemaker.TensorFlow-estimator" data-toc-modified-id="Create-a-training-job-using-the-sagemaker.TensorFlow-estimator-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Create a training job using the sagemaker.TensorFlow estimator</a></span></li><li><span><a href="#Deploy-the-trained-model-to-prepare-for-predictions" data-toc-modified-id="Deploy-the-trained-model-to-prepare-for-predictions-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Deploy the trained model to prepare for predictions</a></span></li><li><span><a href="#Invoking-the-endpoint" data-toc-modified-id="Invoking-the-endpoint-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Invoking the endpoint</a></span></li><li><span><a href="#Deleting-the-endpoint" data-toc-modified-id="Deleting-the-endpoint-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Deleting the endpoint</a></span></li></ul></div>

## Set up the environment

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

## Download the MNIST dataset

In [2]:
import utils
from tensorflow.contrib.learn.python.learn.datasets import mnist
import tensorflow as tf

data_sets = mnist.read_data_sets('data', dtype=tf.uint8, reshape=False, validation_size=5000)

utils.convert_to(data_sets.train, 'train', 'data')
utils.convert_to(data_sets.validation, 'validation', 'data')
utils.convert_to(data_sets.test, 'test', 'data')

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
('Writing', 'data/train.tfrecords')
('Writing', 'data/validation.tfrecords')
('Writing', 'data/test.tfrecords')


## Upload the data

In [3]:
inputs = sagemaker_session.upload_data(path='data', bucket='sagemaker-ml-ps', key_prefix='data/mnist')
inputs

's3://sagemaker-ml-ps/data/mnist'

## Custom function for Tensorflow

In [4]:
!cat 'mnist.py'

import os
import tensorflow as tf
from tensorflow.python.estimator.model_fn import ModeKeys as Modes

INPUT_TENSOR_NAME = 'inputs'
SIGNATURE_NAME = 'predictions'

LEARNING_RATE = 0.001


def model_fn(features, labels, mode, params):
    # Input Layer
    input_layer = tf.reshape(features[INPUT_TENSOR_NAME], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.d

## Create a training job using the sagemaker.TensorFlow estimator

In [5]:
from sagemaker.tensorflow import TensorFlow

mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             training_steps=1000, 
                             evaluation_steps=100,
                             train_instance_count=2,
                             train_instance_type='ml.c4.xlarge')

mnist_estimator.fit(inputs)

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-866319475440
INFO:sagemaker:Creating training-job with name: sagemaker-tensorflow-py2-cpu-2018-03-06-06-36-47-338


....................................................................
executing startup script (first run)
2018-03-06 06:42:23,443 INFO - root - running container entrypoint
2018-03-06 06:42:23,443 INFO - root - starting train task
2018-03-06 06:42:24,982 INFO - botocore.vendored.requests.packages.urllib3.connectionpool - Starting new HTTP connection (1): 169.254.170.2
2018-03-06 06:42:25,884 INFO - botocore.vendored.requests.packages.urllib3.connectionpool - Starting new HTTPS connection (1): s3.amazonaws.com
2018-03-06 06:42:26,006 INFO - botocore.vendored.requests.packages.urllib3.connectionpool - Starting new HTTPS connection (1): s3.amazonaws.com
INFO:tensorflow:----------------------TF_CONFIG--------------------------
INFO:tensorflow:{"environment": "cloud", "cluster": {"worker": ["algo-2:2222"], "ps": ["algo-1:2223", "algo-2:2223"], "master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
INFO:tensorflow:---------------------------------------------------------
INFO:te

INFO:tensorflow:loss = 2.333385, step = 0
2018-03-06 06:43:02.112376: I tensorflow/core/distributed_runtime/master_session.cc:1004] Start master session 74296cad934d7c88 with config: gpu_options { per_process_gpu_memory_fraction: 1 } allow_soft_placement: true
INFO:tensorflow:loss = 0.16479854, step = 82
INFO:tensorflow:global_step/sec: 3.71356
INFO:tensorflow:loss = 0.111868255, step = 117 (29.912 sec)
INFO:tensorflow:global_step/sec: 6.82797
INFO:tensorflow:loss = 0.020259047, step = 274 (28.361 sec)
INFO:tensorflow:global_step/sec: 6.97166
INFO:tensorflow:loss = 0.080218576, step = 329 (30.713 sec)
INFO:tensorflow:global_step/sec: 6.90278
INFO:tensorflow:loss = 0.02698376, step = 465 (27.722 sec)
INFO:tensorflow:global_step/sec: 6.63655
INFO:tensorflow:loss = 0.066274956, step = 534 (30.355 sec)
INFO:tensorflow:global_step/sec: 6.67109
INFO:tensorflow:loss = 0.026596665, step = 660 (28.924 sec)
INFO:tensorflow:global_step/sec: 6.91668
INFO:tensorflow:loss = 0.045887195, step = 744 (

## Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint which serves prediction requests in real-time.

In [6]:
mnist_predictor = mnist_estimator.deploy(initial_instance_count=1,
                                             instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-tensorflow-py2-cpu-2018-03-06-06-36-47-338
INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-py2-cpu-2018-03-06-06-36-47-338


---------------------------------------------------------------------------------------------------!

## Invoking the endpoint

In [7]:
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

for i in range(10):
    data = mnist.test.images[i].tolist()
    tensor_proto = tf.make_tensor_proto(values=np.asarray(data), shape=[1, len(data)], dtype=tf.float32)
    predict_response = mnist_predictor.predict(tensor_proto)
    
    print("========================================")
    label = np.argmax(mnist.test.labels[i])
    print("label is {}".format(label))
    prediction = predict_response['outputs']['classes']['int64Val'][0]
    print("prediction is {}".format(prediction))

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
label is 7
prediction is 7
label is 2
prediction is 2
label is 1
prediction is 1
label is 0
prediction is 0
label is 4
prediction is 4
label is 1
prediction is 1
label is 4
prediction is 4
label is 9
prediction is 9
label is 5
prediction is 5
label is 9
prediction is 9


## Deleting the endpoint

In [8]:
sagemaker.Session().delete_endpoint(mnist_predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: sagemaker-tensorflow-py2-cpu-2018-03-06-06-36-47-338
